# Importing Data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

# file_path = '/content/drive/MyDrive/NNDL_HW2/HW2_Dataset/Train/'
# file1_path = '/content/drive/MyDrive/NNDL_HW2/HW2_Dataset/Train/Cats'
# file2_path = '/content/drive/MyDrive/NNDL_HW2/HW2_Dataset/Train/Dogs'

# train_path = '/content/drive/MyDrive/NNDL_HW2_After_Augment/'
train_path = '/content/drive/MyDrive/NNDL_HW2_After_Augment/'

# test_path = '/content/drive/MyDrive/NNDL_HW2_Before_Augment/HW2_Dataset/Test/'
test_path = '/content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/cats'
test_path2 = '/content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/dogs'

files1 = [f for f in os.listdir(train_path) if os.path.isfile(os.path.join(train_path, f))]
files2 = [f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f))]

test_files = [f for f in os.listdir(test_path) if os.path.isfile(os.path.join(test_path, f))]

print(f"Number of files in {train_path} After Augmentation : {len(files1)}")
print(f"Number of files in {test_path} Before Augmentation : {len(files2)}")
print('-----------------------------------------------')
print(f'Number of files in {test_path} : {len(test_files)}')

# df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of files in /content/drive/MyDrive/NNDL_HW2_After_Augment/ After Augmentation : 2166
Number of files in /content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/ Before Augmentation : 0
-----------------------------------------------
Number of files in /content/drive/MyDrive/NNDL_HW2_Before_Augmentation/HW2_Dataset/Test/ : 0


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define hyperparameters
initial_lr = 0.001
lr_decay_rate = 0.1
momentum = 0.9
batch_size = 32
epochs = 10

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Replace the FC layers with new ones for binary classification (dog vs cat)
x = Flatten()(base_model.output)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Freeze all layers except the new FC layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
opt = SGD(learning_rate=initial_lr, momentum=momentum)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation for training images
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Splitting the dataset into training and validation


train_generator = train_datagen.flow_from_directory(file_path,
                                                    target_size=(224, 224),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    subset='training')  # Training set

validation_generator = train_datagen.flow_from_directory(file_path,
                                                         target_size=(224, 224),
                                                         batch_size=batch_size,
                                                         class_mode='categorical',
                                                         subset='validation')  # Validation set

# Train the model with training and validation data
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

# Unfreeze the last CONV block
for layer in model.layers[:15]:
    layer.trainable = False

# Recompile the model to apply the changes
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

all_test_path = '/content/drive/MyDrive/NNDL_HW2/HW2_Dataset/Test/'

# Test data generator for evaluation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(all_test_path ,
                                                  target_size=(224, 224),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

# Evaluate the model on the test set
model.evaluate(test_generator)

94765736/94765736 [==============================] - 1s 0us/step


NameError: name 'file_path' is not defined